***This project is implimented on Google Colab***




***Google colab is a cloud based data science work space similar to the jupyter notebook. Each colab session is equipped with a virtual machine running 13 GB of ram and either a CPU, GPU, or TPU processor.***

***The only pre-requicite for working on Google colab is creating a google account***

***For using colab the following website can be used https://colab.research.google.com.***

***GUP was selected as the runtime type for faster execution***

**Importing Important Libraries**

In [0]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import cross_val_score
import pandas_profiling

In [0]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

**Uploading Data to Colab**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data (1).csv


In [0]:
import io
raw_data = pd.read_csv(io.BytesIO(uploaded['data.csv']))

***Random Down-Sampling***

In [0]:
shuffled_data = raw_data.sample(frac=1,random_state=4)

# Put all the fraud class in a separate dataset.
fraud_data = shuffled_data.loc[shuffled_data['fraudulent'] == 't']

#Randomly select 866 observations from the non-fraud (majority class)
non_fraud_data = shuffled_data.loc[shuffled_data['fraudulent'] == 'f'].sample(n=866,random_state=42)

# Concatenate both dataframes again
data = pd.concat([fraud_data, non_fraud_data])

***Taking all catagorical variables in one data frame***

In [0]:
catagorical_data = data.drop(['company_profile', 'description', 'requirements', 'benefits', 'in_balanced_dataset', 'title',	'location',	'department',	'salary_range'], axis=1)

***Encoding catagorical Variables For performing Exploratory Data Analysis***

In [0]:
catagorical_data = catagorical_data.dropna()
from sklearn.preprocessing import LabelEncoder
#lable encoding for catagorical data
bin_cols = catagorical_data.nunique()[catagorical_data.nunique() >= 2].keys().tolist()
le = LabelEncoder()
for i in bin_cols :
    catagorical_data[i] = le.fit_transform(catagorical_data[i])

***Using Pandas Profiling For Exploratory Data Analysis (EDA)***

In [0]:
pandas_profiling.ProfileReport(catagorical_data)

Number of variables,10
Number of observations,720
Total Missing (%),0.0%
Total size in memory,56.3 KiB
Average record size in memory,80.1 B
Numeric,6
Categorical,0
Boolean,4
Date,0
Text (Unique),0
Rejected,0


***Removing Unwanted Columns from the raw data***

In [0]:
data = data.drop(["location", "salary_range", "industry", "function", "title", "department", "salary_range", "telecommuting", "has_company_logo", "has_questions", "employment_type", "required_experience", "required_education", "industry", "function", "in_balanced_dataset"], axis=1)
data = data.reset_index()

**Label Encoding Dependent Variable**

In [0]:
#lable encoding for catagorical data
from sklearn.preprocessing import LabelEncoder
#lable encoding for catagorical data
bin_cols = data.nunique()[data.nunique() == 2].keys().tolist()
le = LabelEncoder()
for i in bin_cols :
    data[i] = le.fit_transform(data[i])

**Cleaning The Text Data**



Regular Expression is used for only keeping letters and removing all special charecters and numbers

lower() is used to normalise the words to lower case

WordNetLemmatizer() is used to lemmatize the words back to its normal form

stopwords.words is used for removing english stop words

In [0]:
# Cleaning the texts
corpus = []
i = 0
description = ""
for i in range(0, 1732):
    description = re.sub('[^a-zA-Z]', ' ', data['description'][i])
    description = description.lower()
    description = description.split()
    wl = WordNetLemmatizer()
    description = [wl.lemmatize(word) for word in description if not word in set(stopwords.words('english'))]
    description = ' '.join(description)
    corpus.append(description)

**Creating a Function for Evaluation**

In [0]:
def model_report(y_act, y_pred):
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, cohen_kappa_score, roc_curve,auc
    print("Accuracy = ", accuracy_score(y_act, y_pred))
    print("Precision = " ,precision_score(y_act, y_pred))
    print("Recall = " ,recall_score(y_act, y_pred))
    print("F1 Score = " ,f1_score(y_act, y_pred))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_act, y_pred)
    print("AUC Score =", auc(false_positive_rate, true_positive_rate))
    print("Kappa score = ",cohen_kappa_score(y_act,y_pred))
    print("Error rate = " ,1 - accuracy_score(y_act, y_pred))
    pass

**Creating Function for Uni-Gram and machine learning models**

In [0]:
def gram_1():
  
    #Creating Uni-Gram Bag-of-words using CountVectorizer
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer(ngram_range=(1,1))
    X = cv.fit_transform(corpus).toarray()
    
    #Storing the dependent Variable in y
    y = data.iloc[:, -1].values
    
    #Chi-Square Feature Selction Technique
    from sklearn.feature_selection import SelectKBest 
    from sklearn.feature_selection import chi2 
      
    # 8500 features with highest chi-squared statistics are selected 
    chi2_features = SelectKBest(chi2, k = 8500)
    X = chi2_features.fit_transform(X, y)
    
    #Splitting the data into train test spit
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 50)
    
    #Finding the distribution of the data
    pdf1 = X[1]
    counts, bin_edges = np.histogram(pdf1, bins = 10, density = True);
    pdf = counts/sum(counts);
    cdf = np.cumsum(pdf);
    plt.plot(bin_edges[1:], pdf);
    plt.plot(bin_edges[1:], cdf);
    
    #Importing Time for clalculating the training and testing time
    import time
    start_time = time.time()
    
    #importing Gaussian Naive Bayes
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = gnb.predict(X_test)
    
    #Getting the predicting probability
    predict_probab = gnb.predict_proba(X_test)
    
    #Geting the log_loss value of the model
    from sklearn.metrics import log_loss
    logloss = log_loss(y_test, predict_probab)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    from sklearn.model_selection import cross_val_score
    accuracy_gausian = cross_val_score(estimator = gnb, X = X_train, y = y_train, cv =10)
    gnb_mean = accuracy_gausian.mean()
    print("10 Fold Cross Validation Score Of Gaussian Naive Bayes:", gnb_mean)
    print('Time Elapsed:', eta)
    print('Log Loss:', logloss)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    start_time = time.time()
    #importing Multinomial Naive Bayes
    from sklearn.naive_bayes import MultinomialNB
    mnb = MultinomialNB()
    mnb.fit(X_train, y_train)
    
    #Getting the predicting probability
    predict_probab = mnb.predict_proba(X_test)
    
    #Geting the log_loss value of the model
    logloss = log_loss(y_test, predict_probab)
    
    # Predicting the Test set results
    y_pred = mnb.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy_gausian = cross_val_score(estimator = mnb, X = X_train, y = y_train, cv =10)
    mnb_mean = accuracy_gausian.mean()
    print("10 Fold Cross Validation Score Of Multinomial Navie Bayes:", mnb_mean)
    print('Time Elapsed:', eta)
    print('log loss:', logloss)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #impoting LightGBM
    import lightgbm as lgb
    d_train = lgb.Dataset(X_train, label=y_train)
    params = {}
    params['learning_rate'] = 0.04
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'binary'
    params['metric'] = 'accuracy'
    params['sub_feature'] = 0.5
    params['num_leaves'] = 30
    params['min_data'] = 5
    params['max_depth'] = 8
    params['subsample'] = 0.7000000000000001
    params['random_state'] = 501
    params['min_split_gain'] = 0.4
    params['min_data_in_leaf'] = 1 
    params['colsample_bytree'] = 0.2
    lgbm = lgb.train(params, d_train, 500)
    
    #Prediction
    y_pred=lgbm.predict(X_test)
    
    #LightGBM gives a continues value for the predicting class.
    #For loop is used for converting the continious predicted value.
    for i in range(0,347):
        if y_pred[i]>=.5:       # setting threshold to .5
            y_pred[i]=1
        else:  
            y_pred[i]=0
    end_time = time.time()
    eta = end_time - start_time
    print('Time Elapsed For LightGBM (with parameter tuning):', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    d_train = lgb.Dataset(X_train, label=y_train)
    params = {}
    params['boosting_type'] = 'gbdt'
    params['metric'] = 'accuracy'
    params['random_state'] = 501
    lgbm = lgb.train(params, d_train, 500)
    #Prediction
    y_pred=lgbm.predict(X_test)
    
    #LightGBM gives a continues value for the predicting class.
    #For loop is used for converting the continious predicted value.   
    for i in range(0,347):
        if y_pred[i]>=.5:       # setting threshold to .5
            y_pred[i]=1
        else:  
            y_pred[i]=0
    end_time = time.time()
    eta = end_time - start_time
    print('LightGBM without paramter tuning')
    print('Time Elapsed For LightGBM:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #importing XGBoost Classifier
    from xgboost import XGBClassifier
    
    #gup_hist is set for working with the GUP 
    xgb = XGBClassifier(subsample = 1, n_estimators = 71, min_child_weight = 1, max_depth = 3, learning_rate = 1.0, gamma = 0.5, eta = 0.060000000000000005, colsample_bytree = 0.6, reg_alpha=1, tree_method = 'gpu_hist')
    xgb.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = xgb.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10)
    xgb_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of XGBOOST(With Paramter Tuning):", xgb_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    

    start_time = time.time()
    #gup_hist is set for working with the GUP 
    xgb1 = XGBClassifier(tree_method = 'gpu_hist')
    
    xgb1.fit(X_train, y_train)   
    # Predicting the Test set results
    y_pred = xgb1.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10)
    xgb_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of XGBOOST(Without Paramter Tuning):", xgb_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #importing Random Forest
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(n_estimators = 910, min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto', max_depth = 100, bootstrap = False)
    rf.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    from sklearn.model_selection import cross_val_score
    accuracy = cross_val_score(estimator = rf, X = X_train, y = y_train, cv =10)
    rf_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of Random Forest(With Paramter Tuning):", rf_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = rf, X = X_train, y = y_train, cv =2)
    rf_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of Random Forest(Without Paramter Tuning):", rf_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)

***Calling the Uni-Gram Function and Printing the Output***

In [0]:
gram_1()

10 Fold Cross Validation Score Of Gaussian Naive Bayes: 0.8701104746161814
Time Elapsed: 0.40948057174682617
Log Loss: 4.777698175664878
Accuracy =  0.861671469740634
Precision =  0.9779411764705882
Recall =  0.7471910112359551
F1 Score =  0.8471337579617835
AUC Score = 0.8647197659730071
Kappa score =  0.7248852036602689
Error rate =  0.13832853025936598
10 Fold Cross Validation Score Of Multinomial Navie Bayes: 0.8642082715928178
Time Elapsed: 0.1016085147857666
log loss: 1.537924800031964
Accuracy =  0.8530259365994236
Precision =  0.9150326797385621
Recall =  0.7865168539325843
F1 Score =  0.8459214501510575
AUC Score = 0.8547968885047538
Kappa score =  0.7069499412144595
Error rate =  0.1469740634005764
Time Elapsed For LightGBM (with parameter tuning): 2.2443697452545166
Accuracy =  0.8559077809798271
Precision =  0.8636363636363636
Recall =  0.8539325842696629
F1 Score =  0.8587570621468926
AUC Score = 0.855960374975068
Kappa score =  0.711707819613838
Error rate =  0.1440922190

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


10-Fold Cross  Validation Score of Random Forest(Without Paramter Tuning): 0.7862773065085205
Time Elapsed: 0.33048295974731445
Accuracy =  0.8357348703170029
Precision =  0.8757763975155279
Recall =  0.7921348314606742
F1 Score =  0.831858407079646
AUC Score = 0.836895818097201
Kappa score =  0.6720824974716912
Error rate =  0.1642651296829971


**Creating Function for Bi-Gram and machine learning models**

In [0]:
def gram_2():
    #Creating Bag of words model with Bi-Gram
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer(ngram_range=(2,2))
    X = cv.fit_transform(corpus).toarray()
    
    #Storing the dependent Variable in y
    y = data.iloc[:, -1].values
    
    from sklearn.feature_selection import SelectKBest 
    from sklearn.feature_selection import chi2   
    # 80000 features with highest chi-squared statistics are selected 
    chi2_features = SelectKBest(chi2, k = 80000)
    X = chi2_features.fit_transform(X, y)
    
    #Creating Train Test Split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 50)
    
    #Importing Time for clalculating the training and testing time
    import time
    start_time = time.time()
    
    #importing Gaussian Naive Bayes
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = gnb.predict(X_test)
    
    #Getting the predicting probability
    predict_probab = gnb.predict_proba(X_test)
    
    #Geting the log_loss value of the model
    from sklearn.metrics import log_loss
    logloss = log_loss(y_test, predict_probab)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    from sklearn.model_selection import cross_val_score
    accuracy_gausian = cross_val_score(estimator = gnb, X = X_train, y = y_train, cv =10)
    gnb_mean = accuracy_gausian.mean()
    print("10 Fold Cross Validation Score Of Gaussian Naive Bayes:", gnb_mean)
    print('Time Elapsed:', eta)
    print('Log Loss:', logloss)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    start_time = time.time()
    #importing Multinomial Naive Bayes
    from sklearn.naive_bayes import MultinomialNB
    mnb = MultinomialNB()
    mnb.fit(X_train, y_train)
    
    #Getting the predicting probability
    predict_probab = mnb.predict_proba(X_test)
    
    #Geting the log_loss value of the model
    logloss = log_loss(y_test, predict_probab)
    
    # Predicting the Test set results
    y_pred = mnb.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy_gausian = cross_val_score(estimator = mnb, X = X_train, y = y_train, cv =10)
    mnb_mean = accuracy_gausian.mean()
    print("10 Fold Cross Validation Score Of Multinomial Navie Bayes:", mnb_mean)
    print('Time Elapsed:', eta)
    print('log loss:', logloss)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #impoting LightGBM
    import lightgbm as lgb
    d_train = lgb.Dataset(X_train, label=y_train)
    params = {}
    params['learning_rate'] = 0.04
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'binary'
    params['metric'] = 'accuracy'
    params['sub_feature'] = 0.5
    params['num_leaves'] = 30
    params['min_data'] = 5
    params['max_depth'] = 8
    params['subsample'] = 0.7000000000000001
    params['random_state'] = 501
    params['min_split_gain'] = 0.4
    params['min_data_in_leaf'] = 1 
    params['colsample_bytree'] = 0.2
    lgbm = lgb.train(params, d_train, 500)
    
    #Prediction
    y_pred=lgbm.predict(X_test)
    
    #LightGBM gives a continues value for the predicting class.
    #For loop is used for converting the continious predicted value.
    for i in range(0,347):
        if y_pred[i]>=.5:       # setting threshold to .5
            y_pred[i]=1
        else:  
            y_pred[i]=0
    end_time = time.time()
    eta = end_time - start_time
    print('Time Elapsed For LightGBM (with parameter tuning):', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    d_train = lgb.Dataset(X_train, label=y_train)
    params = {}
    params['boosting_type'] = 'gbdt'
    params['metric'] = 'accuracy'
    params['random_state'] = 501
    lgbm = lgb.train(params, d_train, 500)
    #Prediction
    y_pred=lgbm.predict(X_test)
    
    #LightGBM gives a continues value for the predicting class.
    #For loop is used for converting the continious predicted value.   
    for i in range(0,347):
        if y_pred[i]>=.5:       # setting threshold to .5
            y_pred[i]=1
        else:  
            y_pred[i]=0
    end_time = time.time()
    eta = end_time - start_time
    print('LightGBM without paramter tuning')
    print('Time Elapsed For LightGBM:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #importing XGBoost Classifier
    from xgboost import XGBClassifier
    
    #gup_hist is set for working with the GUP 
    xgb = XGBClassifier(subsample = 1, n_estimators = 71, min_child_weight = 1, max_depth = 3, learning_rate = 1.0, gamma = 0.5, eta = 0.060000000000000005, colsample_bytree = 0.6, reg_alpha=1, tree_method = 'gpu_hist')
    xgb.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = xgb.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10)
    xgb_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of XGBOOST(With Paramter Tuning):", xgb_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    

    start_time = time.time()
    #gup_hist is set for working with the GUP 
    xgb1 = XGBClassifier(tree_method = 'gpu_hist')
    
    xgb1.fit(X_train, y_train)   
    # Predicting the Test set results
    y_pred = xgb1.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10)
    xgb_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of XGBOOST(Without Paramter Tuning):", xgb_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #importing Random Forest
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(n_estimators = 910, min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto', max_depth = 100, bootstrap = False)
    rf.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    from sklearn.model_selection import cross_val_score
    accuracy = cross_val_score(estimator = rf, X = X_train, y = y_train, cv =10)
    rf_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of Random Forest(With Paramter Tuning):", rf_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = rf, X = X_train, y = y_train, cv =2)
    rf_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of Random Forest(Without Paramter Tuning):", rf_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)

***Calling the Bi-Gram Function and Printing the Output***

In [0]:
gram_2()

10 Fold Cross Validation Score Of Gaussian Naive Bayes: 0.9653419758639294
Time Elapsed: 3.864198923110962
Log Loss: 1.2939599225759053
Accuracy =  0.962536023054755
Precision =  0.9714285714285714
Recall =  0.9550561797752809
F1 Score =  0.9631728045325779
AUC Score = 0.9627351904793563
Kappa score =  0.9250552408167334
Error rate =  0.03746397694524495
10 Fold Cross Validation Score Of Multinomial Navie Bayes: 0.9191361350779739
Time Elapsed: 0.9399223327636719
log loss: 0.8268298882139227
Accuracy =  0.9135446685878963
Precision =  0.9567901234567902
Recall =  0.8707865168539326
F1 Score =  0.911764705882353
AUC Score = 0.9146831992553687
Kappa score =  0.8273860847648736
Error rate =  0.08645533141210371
Time Elapsed For LightGBM (with parameter tuning): 30.0747287273407
Accuracy =  0.8357348703170029
Precision =  0.9115646258503401
Recall =  0.7528089887640449
F1 Score =  0.8246153846153846
AUC Score = 0.837942955920484
Kappa score =  0.6727660771305197
Error rate =  0.16426512968

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


10-Fold Cross  Validation Score of Random Forest(Without Paramter Tuning): 0.7682231063733954
Time Elapsed: 5.513104438781738
Accuracy =  0.7867435158501441
Precision =  0.7708333333333334
Recall =  0.8314606741573034
F1 Score =  0.8000000000000002
AUC Score = 0.7855528222857523
Kappa score =  0.5723042073353544
Error rate =  0.21325648414985587


**Creating Function for Tri-Gram and machine learning models**

In [0]:
def gram_3():
    #Creating Bag of words model with Tri-Gram
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer(ngram_range=(3,3))
    X = cv.fit_transform(corpus).toarray()
    #Storing the dependent Variable in y
    y = data.iloc[:, -1].values
    
    from sklearn.feature_selection import SelectKBest 
    from sklearn.feature_selection import chi2   
    # 30000 features with highest chi-squared statistics are selected 
    chi2_features = SelectKBest(chi2, k = 30000)
    X = chi2_features.fit_transform(X, y)
    
    #creating Train Test Split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 50)
    
    #Importing Time for clalculating the training and testing time
    import time
    start_time = time.time()
    
    #importing Gaussian Naive Bayes
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = gnb.predict(X_test)
    
    #Getting the predicting probability
    predict_probab = gnb.predict_proba(X_test)
    
    #Geting the log_loss value of the model
    from sklearn.metrics import log_loss
    logloss = log_loss(y_test, predict_probab)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    from sklearn.model_selection import cross_val_score
    accuracy_gausian = cross_val_score(estimator = gnb, X = X_train, y = y_train, cv =10)
    gnb_mean = accuracy_gausian.mean()
    print("10 Fold Cross Validation Score Of Gaussian Naive Bayes:", gnb_mean)
    print('Time Elapsed:', eta)
    print('Log Loss:', logloss)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    start_time = time.time()
    #importing Multinomial Naive Bayes
    from sklearn.naive_bayes import MultinomialNB
    mnb = MultinomialNB()
    mnb.fit(X_train, y_train)
    
    #Getting the predicting probability
    predict_probab = mnb.predict_proba(X_test)
    
    #Geting the log_loss value of the model
    logloss = log_loss(y_test, predict_probab)
    
    # Predicting the Test set results
    y_pred = mnb.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy_gausian = cross_val_score(estimator = mnb, X = X_train, y = y_train, cv =10)
    mnb_mean = accuracy_gausian.mean()
    print("10 Fold Cross Validation Score Of Multinomial Navie Bayes:", mnb_mean)
    print('Time Elapsed:', eta)
    print('log loss:', logloss)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #impoting LightGBM
    import lightgbm as lgb
    d_train = lgb.Dataset(X_train, label=y_train)
    params = {}
    params['learning_rate'] = 0.04
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'binary'
    params['metric'] = 'accuracy'
    params['sub_feature'] = 0.5
    params['num_leaves'] = 30
    params['min_data'] = 5
    params['max_depth'] = 8
    params['subsample'] = 0.7000000000000001
    params['random_state'] = 501
    params['min_split_gain'] = 0.4
    params['min_data_in_leaf'] = 1 
    params['colsample_bytree'] = 0.2
    lgbm = lgb.train(params, d_train, 500)
    
    #Prediction
    y_pred=lgbm.predict(X_test)
    
    #LightGBM gives a continues value for the predicting class.
    #For loop is used for converting the continious predicted value.
    for i in range(0,347):
        if y_pred[i]>=.5:       # setting threshold to .5
            y_pred[i]=1
        else:  
            y_pred[i]=0
    end_time = time.time()
    eta = end_time - start_time
    print('Time Elapsed For LightGBM (with parameter tuning):', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    d_train = lgb.Dataset(X_train, label=y_train)
    params = {}
    params['boosting_type'] = 'gbdt'
    params['metric'] = 'accuracy'
    params['random_state'] = 501
    lgbm = lgb.train(params, d_train, 500)
    #Prediction
    y_pred=lgbm.predict(X_test)
    
    #LightGBM gives a continues value for the predicting class.
    #For loop is used for converting the continious predicted value.   
    for i in range(0,347):
        if y_pred[i]>=.5:       # setting threshold to .5
            y_pred[i]=1
        else:  
            y_pred[i]=0
    end_time = time.time()
    eta = end_time - start_time
    print('LightGBM without paramter tuning')
    print('Time Elapsed For LightGBM:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #importing XGBoost Classifier
    from xgboost import XGBClassifier
    #gup_hist is set for working with the GUP 
    xgb = XGBClassifier(subsample = 1, n_estimators = 71, min_child_weight = 1, max_depth = 3, learning_rate = 1.0, gamma = 0.5, eta = 0.060000000000000005, colsample_bytree = 0.6, reg_alpha=1, tree_method = 'gpu_hist')
    xgb.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = xgb.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10)
    xgb_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of XGBOOST(With Paramter Tuning):", xgb_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    

    start_time = time.time()
    #gup_hist is set for working with the GUP 
    xgb1 = XGBClassifier(tree_method = 'gpu_hist')
    xgb1.fit(X_train, y_train)   
    # Predicting the Test set results
    y_pred = xgb1.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10)
    xgb_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of XGBOOST(Without Paramter Tuning):", xgb_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #importing Random Forest
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(n_estimators = 910, min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto', max_depth = 100, bootstrap = False)
    rf.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    from sklearn.model_selection import cross_val_score
    accuracy = cross_val_score(estimator = rf, X = X_train, y = y_train, cv =10)
    rf_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of Random Forest(With Paramter Tuning):", rf_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = rf, X = X_train, y = y_train, cv =2)
    rf_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of Random Forest(Without Paramter Tuning):", rf_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)

***Calling the Tri-Gram Function and Printing the Output***

In [0]:
gram_3()

10 Fold Cross Validation Score Of Gaussian Naive Bayes: 0.929260514152943
Time Elapsed: 1.4485790729522705
Log Loss: 3.085596738450234
Accuracy =  0.9106628242074928
Precision =  0.9622641509433962
Recall =  0.8595505617977528
F1 Score =  0.9080118694362018
AUC Score = 0.9120238016089356
Kappa score =  0.8217121074003481
Error rate =  0.08933717579250722
10 Fold Cross Validation Score Of Multinomial Navie Bayes: 0.8837479556183678
Time Elapsed: 0.363375186920166
log loss: 1.8334549117144194
Accuracy =  0.8386167146974063
Precision =  0.9485294117647058
Recall =  0.7247191011235955
F1 Score =  0.8216560509554142
AUC Score = 0.8416494913902002
Kappa score =  0.6790327376036471
Error rate =  0.16138328530259372
Time Elapsed For LightGBM (with parameter tuning): 14.962936878204346
Accuracy =  0.7867435158501441
Precision =  0.9262295081967213
Recall =  0.6348314606741573
F1 Score =  0.7533333333333333
AUC Score = 0.7907885114021673
Kappa score =  0.5767455660315157
Error rate =  0.21325648

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


10-Fold Cross  Validation Score of Random Forest(Without Paramter Tuning): 0.823817239279667
Time Elapsed: 3.035680055618286
Accuracy =  0.8357348703170029
Precision =  0.9689922480620154
Recall =  0.702247191011236
F1 Score =  0.8143322475570033
AUC Score = 0.8392892759789908
Kappa score =  0.6736407887138025
Error rate =  0.1642651296829971


**Creating Function for TF-IDF and machine learning models**

In [0]:
def tfidf():
    #creating TF-IDF Vectorizer with Bi-gram
    from sklearn.feature_extraction.text import TfidfVectorizer 
    tfidf_vectorizer=TfidfVectorizer(use_idf=True, ngram_range = (2,2))
    fitted_vectorizer=tfidf_vectorizer.fit(corpus)
    X=fitted_vectorizer.transform(corpus).toarray()
    #Storing the dependent Variable in y
    y = data.iloc[:, -1].values
    
    # Load libraries 
    from sklearn.feature_selection import SelectKBest 
    from sklearn.feature_selection import chi2 
      
    # 35000 features with highest chi-squared statistics are selected 
    chi2_features = SelectKBest(chi2, k = 35000)
    X = chi2_features.fit_transform(X, y) 
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 50)
    
    pdf1 = X[1].reshape(-1, 1)
    counts, bin_edges = np.histogram(pdf1, bins = 10, density = True);
    pdf = counts/sum(counts);
    cdf = np.cumsum(pdf);
    
    #Importing Time for clalculating the training and testing time
    import time
    start_time = time.time()
    
    #importing Gaussian Naive Bayes
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = gnb.predict(X_test)
    
    #Getting the predicting probability
    predict_probab = gnb.predict_proba(X_test)
    
    #Geting the log_loss value of the model
    from sklearn.metrics import log_loss
    logloss = log_loss(y_test, predict_probab)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    from sklearn.model_selection import cross_val_score
    accuracy_gausian = cross_val_score(estimator = gnb, X = X_train, y = y_train, cv =10)
    gnb_mean = accuracy_gausian.mean()
    print("10 Fold Cross Validation Score Of Gaussian Naive Bayes:", gnb_mean)
    print('Time Elapsed:', eta)
    print('Log Loss:', logloss)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    start_time = time.time()
    #importing Multinomial Naive Bayes
    from sklearn.naive_bayes import MultinomialNB
    mnb = MultinomialNB()
    mnb.fit(X_train, y_train)
    
    #Getting the predicting probability
    predict_probab = mnb.predict_proba(X_test)
    
    #Geting the log_loss value of the model
    logloss = log_loss(y_test, predict_probab)
    
    # Predicting the Test set results
    y_pred = mnb.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy_gausian = cross_val_score(estimator = mnb, X = X_train, y = y_train, cv =10)
    mnb_mean = accuracy_gausian.mean()
    print("10 Fold Cross Validation Score Of Multinomial Navie Bayes:", mnb_mean)
    print('Time Elapsed:', eta)
    print('log loss:', logloss)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #impoting LightGBM
    import lightgbm as lgb
    d_train = lgb.Dataset(X_train, label=y_train)
    params = {}
    params['learning_rate'] = 0.04
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'binary'
    params['metric'] = 'accuracy'
    params['sub_feature'] = 0.5
    params['num_leaves'] = 30
    params['min_data'] = 5
    params['max_depth'] = 8
    params['subsample'] = 0.7000000000000001
    params['random_state'] = 501
    params['min_split_gain'] = 0.4
    params['min_data_in_leaf'] = 1 
    params['colsample_bytree'] = 0.2
    lgbm = lgb.train(params, d_train, 500)
    
    #Prediction
    y_pred=lgbm.predict(X_test)
    
    #LightGBM gives a continues value for the predicting class.
    #For loop is used for converting the continious predicted value.
    for i in range(0,347):
        if y_pred[i]>=.5:       # setting threshold to .5
            y_pred[i]=1
        else:  
            y_pred[i]=0
    end_time = time.time()
    eta = end_time - start_time
    print('Time Elapsed For LightGBM (with parameter tuning):', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    d_train = lgb.Dataset(X_train, label=y_train)
    params = {}
    params['boosting_type'] = 'gbdt'
    params['metric'] = 'accuracy'
    params['random_state'] = 501
    lgbm = lgb.train(params, d_train, 500)
    #Prediction
    y_pred=lgbm.predict(X_test)
    
    #LightGBM gives a continues value for the predicting class.
    #For loop is used for converting the continious predicted value.   
    for i in range(0,347):
        if y_pred[i]>=.5:       # setting threshold to .5
            y_pred[i]=1
        else:  
            y_pred[i]=0
    end_time = time.time()
    eta = end_time - start_time
    print('LightGBM without paramter tuning')
    print('Time Elapsed For LightGBM:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #importing XGBoost Classifier
    from xgboost import XGBClassifier
    xgb = XGBClassifier(subsample = 1, n_estimators = 71, min_child_weight = 1, max_depth = 3, learning_rate = 1.0, gamma = 0.5, eta = 0.060000000000000005, colsample_bytree = 0.6, reg_alpha=1, tree_method = 'gpu_hist')
    xgb.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = xgb.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10)
    xgb_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of XGBOOST(With Paramter Tuning):", xgb_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    

    start_time = time.time()
    xgb1 = XGBClassifier(tree_method = 'gpu_hist')
    xgb1.fit(X_train, y_train)   
    # Predicting the Test set results
    y_pred = xgb1.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10)
    xgb_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of XGBOOST(Without Paramter Tuning):", xgb_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    #importing Random Forest
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(n_estimators = 910, min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto', max_depth = 100, bootstrap = False)
    rf.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    from sklearn.model_selection import cross_val_score
    accuracy = cross_val_score(estimator = rf, X = X_train, y = y_train, cv =10)
    rf_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of Random Forest(With Paramter Tuning):", rf_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)
    
    
    start_time = time.time()
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    end_time = time.time()
    eta = end_time - start_time
    
    #10-fold cross validation score
    accuracy = cross_val_score(estimator = rf, X = X_train, y = y_train, cv =2)
    rf_cross = accuracy.mean()
    print("10-Fold Cross  Validation Score of Random Forest(Without Paramter Tuning):", rf_cross)
    print('Time Elapsed:', eta)
    
    #printing the output using the model report function
    model_report(y_test, y_pred)

***Calling the TF-IDF Function and Printing the Output***

In [0]:
tfidf()

10 Fold Cross Validation Score Of Gaussian Naive Bayes: 0.976196281946198
Time Elapsed: 1.5557653903961182
Log Loss: 0.49767689329842585
Accuracy =  0.9855907780979827
Precision =  0.9942857142857143
Recall =  0.9775280898876404
F1 Score =  0.9858356940509916
AUC Score = 0.9858054650621634
Kappa score =  0.9711750926218206
Error rate =  0.014409221902017322
10 Fold Cross Validation Score Of Multinomial Navie Bayes: 0.8945813327123131
Time Elapsed: 0.2414391040802002
log loss: 0.3746874637370203
Accuracy =  0.899135446685879
Precision =  0.9798657718120806
Recall =  0.8202247191011236
F1 Score =  0.8929663608562691
AUC Score = 0.9012366199055913
Kappa score =  0.7990070335126189
Error rate =  0.10086455331412103
Time Elapsed For LightGBM (with parameter tuning): 15.929367542266846
Accuracy =  0.8357348703170029
Precision =  0.9172413793103448
Recall =  0.7471910112359551
F1 Score =  0.8235294117647058
AUC Score = 0.838092547038096
Kappa score =  0.6728634987843403
Error rate =  0.164265

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


10-Fold Cross  Validation Score of Random Forest(Without Paramter Tuning): 0.7805032154743137
Time Elapsed: 2.229353427886963
Accuracy =  0.792507204610951
Precision =  0.7676767676767676
Recall =  0.8539325842696629
F1 Score =  0.8085106382978724
AUC Score = 0.7908716175786183
Kappa score =  0.583488930381435
Error rate =  0.20749279538904897


***Getting the data ready for Hyper-Parameter Tuning***

In [0]:
#creating TF-IDF Vectorizer with Bi-gram
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf_vectorizer=TfidfVectorizer(use_idf=True, ngram_range = (2,2))
fitted_vectorizer=tfidf_vectorizer.fit(corpus)
X=fitted_vectorizer.transform(corpus).toarray()

#Storing the dependent Variable in y
y = data.iloc[:, -1].values

# Load libraries 
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2 
     
# 15000 features with highest chi-squared statistics are selected 
chi2_features = SelectKBest(chi2, k = 35000)
X = chi2_features.fit_transform(X, y)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 50)

***Initializing the XGBoost Classifier***

In [0]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

***Setting the hyper-paramter range for XGBoost***

In [0]:
seed = 345
est = range(1,100,5)
eta = np.arange(0.01,0.2,0.05)
max_dept = range(3,10,1)
sub_sample = np.arange(0.5,1,0.1)
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'n_estimators' : est,
        'learning_rate' : np.linspace(1e-16,1,3),
        'eta' : eta,
        
        }
param_comb = 100

***Initializing the Random Search CV for Hyper-Parameter Tuning***

In [0]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs= -1, verbose=3, random_state=10, cv = 3 )
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 34.2min


***Printing the results optained by Random Search Function***

In [0]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)


 All results:
{'mean_fit_time': array([ 96.29152576, 133.53986398,  73.21192384,  27.35551778,
        22.87895783,   6.10692239,  74.99379222,  33.84919771,
        54.14499013,  79.34299787,  57.05286741,  64.91112288,
       121.76516859,  49.0546906 , 114.57513038,  38.82478952,
        37.18044162,  40.83018152,  22.82832392, 107.75639733,
        36.83959492,   9.96962881,  49.06266348,  97.52881591,
        21.4095606 ,  40.40099716,  39.92762335,  62.97546776,
        68.58057324,  86.04097891,  98.14887834,  15.39529411,
       113.72413564,   5.65984782, 108.45314415,  24.21324587,
        74.82822386,  37.68791946,  90.73205558,  71.9793725 ,
        28.3806142 ,   5.40434798,  70.80809228, 121.56047591,
       104.60585904,  49.89321733,  61.6070385 ,  89.00402538,
       136.00476146,  44.14504155,  46.69174425,  84.1431063 ,
        32.28344703,  24.0236458 , 130.95641597,  40.10376   ,
        80.53896761,  52.68732182,  70.93327332, 138.0437816 ,
        49.45042801,  

***Trying out the optained Hyper-paramters before using in the function***

In [0]:
from xgboost import XGBClassifier
xgb = XGBClassifier(subsample = 1, n_estimators = 71, min_child_weight = 1, max_depth = 3, learning_rate = 1.0, gamma = 0.5, eta = 0.060000000000000005, colsample_bytree = 0.6, reg_alpha=1 )
xgb.fit(X_train, y_train)
# Predicting the Test set results
y_pred = xgb.predict(X_test)

In [0]:
model_report(y_test, y_pred)

Accuracy =  0.7931034482758621
Precision =  0.8235294117647058
Recall =  0.7692307692307693
F1 Score =  0.7954545454545454
AUC Score = 0.794253938832252
Kappa score =  0.5866437904183714
Error rate =  0.2068965517241379


***Initializing the LightGBM classifier***

In [0]:
import lightgbm as lgb
lgbm = lgb.LGBMClassifier()

***Setting the hyper-paramter range for XGBoost***

In [0]:
lgbmParams = {
    'learning_rate': np.arange(0.01,0.05,0.01),
    'num_leaves': range(10,200,5),
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'max_depth' : np.arange(1,10,1),
    'random_state' : [501], 
    'colsample_bytree' : np.arange(0.1,1,0.1),
    'subsample' : np.arange(0.1,1,0.1),
    'min_split_gain' : np.arange(0.1,1,0.1),
    'min_data_in_leaf':range(1,20,1),
    'metric':['accuracy']
    }
param_comb = 100

***Initializing the Random Search CV for Hyper-Parameter Tuning***

In [0]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(lgbm, param_distributions=lgbmParams, n_iter=param_comb, scoring='accuracy', n_jobs= -1, verbose=3, random_state=10 )

In [0]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.2min finished


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=LGBMClassifier(boosting_type='gbdt',
                                            class_weight=None,
                                            colsample_bytree=1.0,
                                            importance_type='split',
                                            learning_rate=0.1, max_depth=-1,
                                            min_child_samples=20,
                                            min_child_weight=0.001,
                                            min_split_gain=0.0,
                                            n_estimators=100, n_jobs=-1,
                                            num_leaves=31, objective=None,
                                            random_state=None, reg_alpha=0....
                                        'metric': ['accuracy'],
                                        'min_data_in_leaf': range(1, 20),
                                 

***Printing the results optained by Random Search Function***

In [0]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)


 All results:
{'mean_fit_time': array([2.19994744, 1.69004679, 1.85898566, 6.58519387, 1.67266925,
       1.14472985, 2.21620568, 1.66314546, 1.92646909, 2.68380841,
       1.16716703, 2.50916306, 1.61363578, 1.14009507, 2.59212462,
       3.24925669, 1.5644753 , 2.72856212, 1.53053252, 2.10428739,
       6.45479226, 1.81308532, 2.31632535, 2.13543391, 1.35669438,
       2.97906582, 2.06694198, 1.84628479, 1.81299305, 1.77346325,
       2.05590741, 2.08662558, 1.14728944, 2.0918204 , 1.70535755,
       1.19861277, 2.3367784 , 5.30071465, 1.73285182, 1.72083489,
       1.90072155, 2.32453148, 1.89130767, 1.42926407, 1.48158868,
       2.09677633, 1.19518781, 1.92468198, 1.97160856, 1.70563356,
       2.3379101 , 1.96550822, 1.59036692, 2.2778151 , 1.35383248,
       1.8788542 , 5.99624896, 1.4814655 , 1.64796901, 2.61944548,
       1.4011693 , 2.20343558, 2.00939099, 4.62887669, 1.96261334,
       2.49212384, 1.26759521, 1.86770964, 1.78331216, 1.16472777,
       2.4027582 , 1.51931294

***Trying out the optained Hyper-paramters before using in the function***

In [0]:
lgbm = lgb.LGBMClassifier(subsample = 0.7000000000000001, random_state = 501, objective= 'binary', num_leaves= 30, min_split_gain = 0.4, min_data_in_leaf = 1, metric = 'accuracy', max_depth = 8, learning_rate = 0.04, colsample_bytree = 0.2, boosting_type = 'gbdt')

In [0]:
lgbm.fit(X_train, y_train)
# Predicting the Test set results
y_pred = lgbm.predict(X_test)
model_report(y_test, y_pred)

Accuracy =  0.8045977011494253
Precision =  0.8607594936708861
Recall =  0.7472527472527473
F1 Score =  0.7999999999999999
AUC Score = 0.8073613133854098
Kappa score =  0.6108406788580449
Error rate =  0.1954022988505747
AUC-ROC Curve: 


***Initializing the Random Forest classifier***

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

***Setting the hyper-paramter range for Random Forest***

In [0]:
from sklearn.model_selection import RandomizedSearchCV
max_features = ['auto', 'sqrt']
n_estimators = range(200, 2000, 10)
max_depth = range(10, 110, 10)
min_samples_split = range(2, 10, 1)
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

***Initializing the Random Search CV for Hyper-Parameter Tuning***

In [0]:
random_search = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 31.8min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 103.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 198.2min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

***Printing the results optained by Random Search Function***

In [0]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)


 All results:
{'mean_fit_time': array([230.27563222, 327.01595434, 412.66071892,  48.76467244,
       313.15084132, 182.99792528, 149.29126946, 351.57185268,
       302.80417061,  23.08409818, 119.65842287, 326.27437814,
        14.40022389,  80.97873696, 138.64048584,  62.38611714,
        24.78761744, 138.8504134 , 227.76396688,  39.55118529,
        54.41166854, 147.63843783, 220.92118994, 285.62994075,
       408.60270166, 113.64148076, 306.80726838, 313.31035177,
        96.72284476,  23.38357965,  58.51733557,  69.54068645,
        91.88440037, 108.50861812, 504.67213027,  44.06780624,
       185.87197741,  38.60177557,  58.43679301,  72.25940196,
       208.62247284,  44.77544204, 134.55331492,  43.41902622,
        52.14710689,  19.18564423, 105.20518398, 105.01435582,
       335.31215382, 102.60989046, 106.44610763, 211.01593781,
       242.65467024, 302.01216197, 204.05701121,  46.54431534,
       262.23889335, 236.64057374, 254.52919006, 144.90901677,
        31.50051324, 2

***Trying out the optained Hyper-paramters before using in the function***

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 910, min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto', max_depth = 100, bootstrap = False)
rf.fit(X_train, y_train)
# Predicting the Test set results
y_pred = rf.predict(X_test)
model_report(y_test, y_pred)

Accuracy =  0.8505747126436781
Precision =  0.9577464788732394
Recall =  0.7472527472527473
F1 Score =  0.839506172839506
AUC Score = 0.8555540844697471
Kappa score =  0.7036551814489715
Error rate =  0.14942528735632188
AUC-ROC Curve: 
